In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Oct 25 12:10:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [5]:
import sys
import numpy as np
import cv2

## Git Clone Darknet

In [6]:
#!git clone https://github.com/pjreddie/darknet #CUDNN 버전 등의 충돌 문제로 컴파일이 되지않음..!
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15494, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 15494 (delta 0), reused 1 (delta 0), pack-reused 15490
Receiving objects: 100% (15494/15494), 14.09 MiB | 24.36 MiB/s, done.
Resolving deltas: 100% (10411/10411), done.


## Compile Darknet using Nvidia GPU

In [7]:
# change makefile to have GPU and OPENCV enabled
%cd /mydrive/darknet-master
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/gdrive/My Drive/darknet-master
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./sr

## make classes.names(클래스 종류), custom_data.data(class, train, test 정보)

In [ ]:
#PYTHON
import os

current_path = os.path.abspath(os.curdir)
COLAB_DARKNET_ESCAPE_PATH = '/mydrive/darknet-master'
COLAB_DARKNET_PATH = '/mydrive/darknet-master'

class_count = 0
YOLO_FORMAT_PATH = current_path + '/custom'
YOLO_IMAGE_PATH = current_path + '/custom'

with open(YOLO_FORMAT_PATH + '/' + 'classes.names', 'w') as names, \
     open(YOLO_FORMAT_PATH + '/' + 'classes.txt', 'r') as txt:
    for line in txt:
        names.write(line)  
        class_count += 1
    print ("[classes.names] is created")

with open(YOLO_FORMAT_PATH + '/' + 'custom_data.data', 'w') as data:
    data.write('classes = ' + str(class_count) + '\n')
    data.write('train = ' + COLAB_DARKNET_ESCAPE_PATH + '/custom/' + 'train.txt' + '\n')
    data.write('valid = ' + COLAB_DARKNET_ESCAPE_PATH + '/custom/' + 'test.txt' + '\n')
    data.write('names = ' + COLAB_DARKNET_ESCAPE_PATH + '/custom/' + 'classes.names' + '\n')
    data.write('backup = backup')
    print ("[custom_data.data] is created")

[classes.names] is created
[custom_data.data] is created


## , train.txt, test.txt(이미지데이터 경로,이름 리스트)

In [ ]:
import random

YOLO_IMAGE_PATH = '/mydrive/drone_dataset/image'
test_percentage = 0.2
paths = []


os.chdir(YOLO_IMAGE_PATH)
for current_dir, dirs, files in os.walk('.'):
    for f in files:
        if f.endswith('.jpg'):
            #image_path = COLAB_DARKNET_PATH + '/custom/' + f
            image_path = YOLO_IMAGE_PATH+'/'+f
            paths.append(image_path + '\n')

# 데이터 500개로 제한 --> train 400 test 100
#paths = paths[:250]+paths[-250:]
#paths_test = paths[:int(len(paths) * test_percentage)]
#paths = paths[int(len(paths) * test_percentage):]
random.shuffle(paths)
paths_test = paths[:int(len(paths) * test_percentage)]
paths = paths[int(len(paths) * test_percentage):]
#paths_test = paths[:50]+paths[-50:]
#paths = paths[50:250]+paths[-250:-50]

with open(YOLO_FORMAT_PATH + '/' + 'train.txt', 'w') as train_txt:
    for path in paths:
        train_txt.write(path)
    print ("[train.txt] is created")

with open(YOLO_FORMAT_PATH + '/' + 'test.txt', 'w') as test_txt:
    for path in paths_test:
        test_txt.write(path)
####     print ("[test.txt] is created")

[train.txt] is created


## image 데이터 custom 안에 넣기

In [ ]:
!cp -p /content/gdrive/MyDrive/jonanja_data/image/* /content/gdrive/MyDrive/darknet-master/custom/

cp: -r not specified; omitting directory '/content/gdrive/MyDrive/jonanja_data/image/darknet'


In [ ]:
!cp -p /content/gdrive/MyDrive/jonanja_data/label/* /content/gdrive/MyDrive/darknet-master/custom/

In [ ]:
print(len(os.listdir('/content/gdrive/MyDrive/darknet-master/custom')))

4659


In [ ]:
!cp -p /content/gdrive/MyDrive/drone_dataset/label/* /content/gdrive/MyDrive/drone_dataset/image/

In [8]:
!mkdir /content/darknet/backup

## lagrange_yolov3-tiny.cfg 변경
* Testing 전부 주석처리
* batch=32, subdivisions=16
* max_batches = 4000 --> (클래스 수 * 2000 정도)
* steps: 3200, 3600 --> max batch size * 0.8, 0.9
* linear 함수 쓰는 두 층의 filters: 21 --> (클래스 수 +5) * 3
* [yolo] class: 2

## load darknet to Colab

In [13]:
%cd darknet

/content/darknet


In [12]:
%cd ..

/content


In [ ]:

if not os.path.exists('darknet'):
    os.makedirs('darknet')
%cd darknet
%ls

## darknet 권한변경

In [14]:
!ls -la '/content/gdrive/My Drive/darknet-master/darknet'
!cp /content/gdrive/My\ Drive/darknet-master/darknet ./darknet
!chmod +x ./darknet

-rwx------ 1 root root 5411920 Oct 25 12:13 '/content/gdrive/My Drive/darknet-master/darknet'


## colab 환경에 custom 복사

In [15]:
!cp -r "/content/gdrive/My Drive/darknet-master/custom" .

## yolov3-tiny.weights에서 yolov3-tiny.conv.15 추출
yolo.weights가 최종단계,   
yolov3일 때 darknet53.conv.74, yolov3-tiny일 때 yolov3-tiny.conv.15가 중간정도 가중치 잡힌 단계

In [16]:
!./darknet partial custom/lagrange_yolov3-tiny.cfg custom/yolov3-tiny.weights custom/yolov3-tiny.conv.15 15

 CUDA-version: 11020 (11020), cuDNN: 8.1.1, GPU count: 1  
 OpenCV version: 3.2.0
 GPU isn't used 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 Create CUDA-stream - 0 
max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256       3 x 3/ 1     26 x  26 x 128 ->   26 x  26 x 256 0.399 BF
   9 max

## yolo-tiny training - 1000epoch마다 모델 저장

In [17]:
!./darknet detector train custom/custom_data.data custom/lagrange_yolov3-tiny.cfg custom/yolov3-tiny.conv.15 -dont_show 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.590431), count: 2, class_loss = 0.619232, iou_loss = 0.426453, total_loss = 1.045684 
 total_bbox = 145961, rewritten_bbox = 0.879687 % 
]2;3903/4000: loss=0.3 hours left=0.0
 3903: 0.349654, 0.417666 avg loss, 0.000010 rate, 0.358365 seconds, 124896 images, 0.026461 hours left
Loaded: 0.000027 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.751829), count: 2, class_loss = 0.008394, iou_loss = 0.215865, total_loss = 0.224258 
 total_bbox = 145963, rewritten_bbox = 0.879675 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.705081), count: 1, class_loss = 0.000244, iou_loss = 0.094609, total_loss = 0.094854 
v3 (mse loss, Normali

In [19]:
from google.colab import files  
files.download('/content/darknet/backup/lagrange_yolov3-tiny_1000.weights')
files.download('/content/darknet/backup/lagrange_yolov3-tiny_2000.weights')
files.download('/content/darknet/backup/lagrange_yolov3-tiny_3000.weights')
files.download('/content/darknet/backup/lagrange_yolov3-tiny_4000.weights')
files.download('/content/darknet/backup/lagrange_yolov3-tiny_final.weights')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
!./darknet detector map custom/custom_data.data custom/lagrange_yolov3-tiny.cfg  backup/lagrange_yolov3-tiny_2000.weights

 CUDA-version: 11020 (11020), cuDNN: 8.1.1, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256   

In [21]:
!./darknet detector map custom/custom_data.data custom/lagrange_yolov3-tiny.cfg  backup/lagrange_yolov3-tiny_3000.weights

 CUDA-version: 11020 (11020), cuDNN: 8.1.1, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256   

In [22]:
!./darknet detector map custom/custom_data.data custom/lagrange_yolov3-tiny.cfg  backup/lagrange_yolov3-tiny_4000.weights

 CUDA-version: 11020 (11020), cuDNN: 8.1.1, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256   

In [23]:
!./darknet detector map custom/custom_data.data custom/lagrange_yolov3-tiny.cfg  backup/lagrange_yolov3-tiny_final.weights

 CUDA-version: 11020 (11020), cuDNN: 8.1.1, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256   